<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/Full_Test_Jacobians_Resnets_SVD_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for generating Full Jacobian SVD over the test data for ResNet data

Can do the full calculation, and a batched (slow, but handles large memory cases)

Can handle: ResnetXXX with CIFAR10, CIFAR100, SVHN

i.e. CIFAR 10
```
 resnet20_cifar10
 resnet56_cifar10
 resnet110_cifar10
 resnet164bn_cifar10
 resnet272bn_cifar10

full_jacobian()


 resnet542bn_cifar10
 resnet1001_cifar10
 resnet1202_cifar10

full_batched_jacobian()

```






In [51]:
Jdir =  '/content/drive/My Drive/J_resnets_fullsvd'

modelnames = ['resnet20_cifar100', 'resnet56_cifar100', 'resnet110_cifar100', 'resnet164bn_cifar100']
modelnames = ['resnet542bn_cifar100']

num_classes = 100
num_data = 10000

batched = True
start_batch = 0
batch_size = 100

device = 'cuda:0'


time: 2.89 ms


In [19]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
time: 2.15 s


In [20]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.17 s


In [21]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.75 ms


In [23]:
!pip install pytorchcv
!pip install powerlaw
!pip install GPUtil
import GPUtil

time: 6.14 s


In [7]:
!import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

import gc


/bin/bash: import: command not found
time: 4.07 s


### Run InstallPhysTheory notebook 

In [8]:
import sys
sys.path.append("/content/drive/My Drive/Jacobian")
import jacobian as jac
import utils

time: 793 ms


In [10]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

time: 90.9 ms


### Compute JJ, in batches or all at once


In [11]:
def get_datasets(thismodel, batch_size):
  print("getting dataset for ", thismodel)
  if thismodel.endswith('cifar100'):
    print("loading cifar100")
    train_dataset = datasets.CIFAR100(
      root='data', 
      train=True, 
      transform=transforms.ToTensor(),
      download=True)
    test_dataset = datasets.CIFAR100(
      root='data', 
      train=False, 
      transform=transforms.ToTensor(),
      download=True) 
 
    train_loader = DataLoader(
      dataset=train_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)
    test_loader = DataLoader(
      dataset=test_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)

  else:
    print("loading cifar10")
    train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

      
  return train_dataset, test_dataset, train_loader, test_loader

time: 7.53 ms


In [12]:
print(thismodel)
train_dataset, test_dataset, train_loader, test_loader = get_datasets(thismodel, batch_size)#utils.get_data(batch_size=batch_size)
#train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

print(test_dataset)
GPUtil.showUtilization()


resnet164bn_cifar100
getting dataset for  resnet164bn_cifar100
loading cifar100


Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
time: 7.04 s


### Hook for model eval later

In [13]:
#model = ptcv_get_model(thismodel, pretrained=True)  

time: 1.21 ms


In [35]:
def read_batched_JJMat(modelname, verbose=False):

  if verbose:
    print("Reading batched JJBlocks for ", modelname)
    
  JJMat = None
  import os, glob
  os.chdir(Jdir)
  for filename in glob.glob("Jfull_{}*".format(modelname)):
    if verbose:
      print(filename) 

    JJBlock = np.loadtxt(filename)
    JJMat = np.vstack([JJMat, JJBlock]) if JJMat is not None else JJBlock

  return JJMat

time: 3.56 ms


In [52]:
import powerlaw

alphas = []
max_evals = []

for modelname in modelnames:
  print(modelname)
  JJMat = read_batched_JJMat(modelname)

  norm = num_classes*3*32*32
  evals = np.linalg.eigh(JJMat)[0]   #LAPAK routine
  evals  = (1.0/ norm)*evals

  fit = powerlaw.Fit(evals)
  alpha = fit.alpha
  D = fit.D
  max_eval = np.max(evals)

  plt.hist(evals, bins=100)
  plt.title("{}  alpha = {:0.2f} max  = {:0.2f} ".format(modelname, alpha, max_eval))
  plt.show()

  print(alpha, D, max_eval)
  alphas.append(alpha)
  max_evals.append(max_eval)



resnet542bn_cifar100


LinAlgError: ignored

time: 21.3 s


In [49]:
!ls *cifar100*

Jfull_resnet110_cifar100__0.csv     Jfull_resnet20_cifar100__32.csv
Jfull_resnet110_cifar100__10.csv    Jfull_resnet20_cifar100__33.csv
Jfull_resnet110_cifar100__11.csv    Jfull_resnet20_cifar100__34.csv
Jfull_resnet110_cifar100__12.csv    Jfull_resnet20_cifar100__35.csv
Jfull_resnet110_cifar100__13.csv    Jfull_resnet20_cifar100__36.csv
Jfull_resnet110_cifar100__14.csv    Jfull_resnet20_cifar100__37.csv
Jfull_resnet110_cifar100__15.csv    Jfull_resnet20_cifar100__38.csv
Jfull_resnet110_cifar100__16.csv    Jfull_resnet20_cifar100__39.csv
Jfull_resnet110_cifar100__17.csv    Jfull_resnet20_cifar100__3.csv
Jfull_resnet110_cifar100__18.csv    Jfull_resnet20_cifar100__40.csv
Jfull_resnet110_cifar100__19.csv    Jfull_resnet20_cifar100__41.csv
Jfull_resnet110_cifar100__1.csv     Jfull_resnet20_cifar100__42.csv
Jfull_resnet110_cifar100__20.csv    Jfull_resnet20_cifar100__43.csv
Jfull_resnet110_cifar100__21.csv    Jfull_resnet20_cifar100__44.csv
Jfull_resnet110_cifar100__22.csv    Jfull_resnet2

In [53]:
JJMat.shape

(1700, 10000)

time: 3.26 ms
